In [1]:
from WirelessSystem.system import *

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import keras

Using TensorFlow backend.


In [4]:
import os
import numpy as np
from SigGAN.train import Trainer

In [5]:
import tensorflow as tf

In [6]:
def norm(sig_u):
    pwr = np.sqrt(np.mean(np.sum(sig_u**2,axis = -1),axis = -1))
    sig_u = sig_u/pwr[:,None,None]
    print(sig_u.shape)
    return sig_u

In [7]:
def shuffle(vec1,vec2,seed = 0):
    np.random.seed(0)
    shfl_indx = np.arange(vec1.shape[0])
    np.random.shuffle(shfl_indx)
    shfl_indx = shfl_indx.astype('int')
    vec1 = vec1[shfl_indx]
    vec2 = np.copy(vec2[shfl_indx])
    return vec1,vec2

In [8]:
def split(vec,n_train,n_valid,n_test):
    vec_train = vec[0:n_train]
    vec_valid = vec[n_train:n_train+n_valid]
    vec_test = vec[n_train+n_valid:]
    return vec_train,vec_valid,vec_test

In [9]:
n_symbols = 256 // 2
n_authorized = 10
n_unauthorized = 0
sess_name = 'test_transmitters_snr'
snr=20
suffix='_t_%d_snr_%d'%(n_authorized,snr)

In [10]:
suffix

'_t_10_snr_20'

In [11]:
#rf_system = RFSystem(n_authorized = n_authorized, n_unauthorized = n_unauthorized, snr = snr, sess_name = sess_name, suffix=suffix)

In [12]:
rf_system = RFSystem(sess_dir=sess_name, suffix=suffix)

In [13]:
sig_auth_, txid_auth = rf_system.get_n_received_symbol_blocks(5000, n_symbols, authorized = 0)
sig_unauth_, txid_unauth = rf_system.get_n_received_symbol_blocks(5000, n_symbols, authorized = 1)
sig_impersonate_, _ = rf_system.get_n_received_symbol_blocks(10, n_symbols, authorized = 2)

In [14]:
sig_auth=sig_auth_
sig_unauth=sig_unauth_

In [15]:
sig_impersonate_ad_, _ = rf_system.get_n_received_symbol_blocks(5000, n_symbols, authorized = 2)
sig_impersonate_ad=sig_impersonate_ad_

In [16]:
sig_impersonate=sig_impersonate_.reshape((-1,2))

In [17]:
# hyper parameters
B = 64 # batch size
T = 256 # Max length of sentence
g_H = 100 # Generator LSTM hidden size
g_lr = 1e-3
d_dropout = 0.5 # dropout ratio
d_lr = 1e-3

n_sample=1 # Number of Monte Calro Search
generate_samples = 2000 # Number of generated sentences

# Pretraining parameters
g_pre_lr = 1e-3
d_pre_lr = 1e-3
g_pre_epochs= 60
d_pre_epochs = 1

In [18]:
trainer = Trainer(rf_system, B, T, n_authorized, g_H, d_dropout, g_lr=g_lr, d_lr=d_lr, n_sample=n_sample, generate_samples=generate_samples)






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [19]:
sig_rd = np.concatenate([sig_auth,sig_impersonate_ad])
txid_rd = np.concatenate([txid_auth,np.ones((sig_impersonate_ad.shape[0],))*n_authorized])

In [20]:
# sig_rd = np.concatenate([sig_auth,sig_unauth])
# txid_rd = np.concatenate([txid_auth,np.ones((sig_unauth.shape[0],))*n_authorized])

In [21]:
sig_rd, txid_rd = shuffle(sig_rd, txid_rd)

In [22]:
txid_disc = txid_rd == n_authorized
txid_disc = np.invert(txid_disc)
txid_disc = txid_disc.astype(int)

In [23]:
print(np.unique(txid_disc))
print(np.unique(txid_rd))

[0 1]
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]


In [24]:
test_frac = 0.1
valid_frac  = 0.2

n_samples  = sig_rd.shape[0]

n_test = int(test_frac*n_samples)
n_valid = int(valid_frac*n_samples)
n_train = n_samples - n_test - n_valid

In [25]:
sig_rd_train,sig_rd_valid,sig_rd_test=split(sig_rd,n_train,n_valid,n_test)

In [26]:
out_train,out_valid,out_test=split(txid_disc,n_train,n_valid,n_test)

In [27]:
trainer.load_pre_train_d(rf_system.full_sess_dir, suffix)

In [28]:
#trainer.pre_train_discriminator(d_epochs=100, d_pre_sig_train=sig_rd_train, d_pre_out_train=out_train, d_pre_sig_valid=sig_rd_valid, d_pre_out_valid=out_valid, lr=1e-5)

In [29]:
#trainer.save_pre_train_d(rf_system.full_sess_dir, suffix=suffix)

In [30]:
print(trainer.test_discriminator(sig_rd_train, out_train))

0.9155714285714286


In [31]:
print(trainer.test_discriminator(sig_rd_valid, out_valid))

0.9095


In [32]:
print(trainer.test_discriminator(sig_rd_test, out_test))

0.926


In [33]:
print(trainer.test_discriminator(sig_impersonate_ad, np.zeros((sig_impersonate_ad.shape[0],))))

0.9948


In [34]:
np.mean(trainer.predict_discriminator(sig_auth))

0.85650325

In [35]:
np.mean(trainer.predict_discriminator(sig_impersonate_ad))

0.14890347

In [36]:
n_samples_im  = sig_impersonate.shape[0]

n_test_im = int(test_frac*n_samples_im)
n_valid_im = int(valid_frac*n_samples_im)
n_train_im = n_samples_im - n_test_im - n_valid_im

In [37]:
sig_im_train,sig_im_valid,sig_im_test=split(sig_impersonate,n_train_im,n_valid_im,n_test)

In [38]:
sig_im_train_re = sig_im_train.reshape((-1, 1, 2))
sig_im_train_re_t = sig_im_train.reshape((-1, 2))

In [39]:
sig_im_valid_re = sig_im_valid.reshape((-1, 1, 2))
sig_im_valid_re_t = sig_im_valid.reshape((-1, 2))

In [40]:
sig_im_test_re = sig_im_test.reshape((-1, 1,  2))

In [41]:
trainer.pre_train_generator(g_epochs=100, g_pre_data_train=[sig_im_train_re, sig_im_train_re_t], g_pre_data_valid=[sig_im_valid_re, sig_im_valid_re_t], lr=1e-3)


Generator pre-training
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              (None, None, 2)      0                                            
__________________________________________________________________________________________________
LSTM (LSTM)                     (None, 100)          41200       Input[0][0]                      
__________________________________________________________________________________________________
DenseMean (Dense)               (None, 2)            202         LSTM[0][0]                       
__________________________________________________________________________________________________
DenseVariance (Dense)           (None, 2)            202         LSTM[0][0]                       
_____________________________________________________________________________________

In [42]:
sig_im_test_re[0]

array([[ 1.0817413, -0.996754 ]], dtype=float32)

In [43]:
print(trainer.predict_pre_generator(sig_im_test_re[:1].reshape(1,1,2)))

[[ 1.0778791 -0.9938676]]


In [44]:
print(trainer.predict_generator(sig_im_test_re[:1].reshape(1,1,2)))

[[ 1.0763426  -0.99353707]]


In [45]:
print(trainer.predict_beta_generator(sig_im_test_re[:1].reshape(1,1,2)))

[[ 1.0815829  -0.99369425]]


In [46]:
print(trainer.predict_curr_discriminator())

0.14950863


In [47]:
trainer.reset_generator(lr=1e-3, epsilon=0.08, beta=15000, binary_reward=True)

In [48]:
trainer.reflect_pre_train()

In [ ]:
trainer.train(steps=1000)

Initial Disc Accuracy: 0.105
0: Disc Batch Accuracy: 0.000
0, 1: Disc Accuracy: 0.255, Average reward: 0.000
1, 1: Disc Accuracy: 0.175, Average reward: 0.000
2, 1: Disc Accuracy: 0.499, Average reward: 0.000
3, 1: Disc Accuracy: 0.862, Average reward: 0.000
4, 1: Disc Accuracy: 0.828, Average reward: 1.000
5, 1: Disc Accuracy: 0.970, Average reward: 1.000
6, 1: Disc Accuracy: 0.972, Average reward: 1.000
7, 1: Disc Accuracy: 0.998, Average reward: 1.000
8, 1: Disc Accuracy: 0.995, Average reward: 1.000
9, 1: Disc Accuracy: 0.999, Average reward: 1.000
10, 1: Disc Accuracy: 1.000, Average reward: 1.000
11, 1: Disc Accuracy: 1.000, Average reward: 1.000
12, 1: Disc Accuracy: 1.000, Average reward: 1.000
13, 1: Disc Accuracy: 1.000, Average reward: 1.000
14, 1: Disc Accuracy: 1.000, Average reward: 1.000
15, 1: Disc Accuracy: 1.000, Average reward: 1.000
16, 1: Disc Accuracy: 1.000, Average reward: 1.000
17, 1: Disc Accuracy: 1.000, Average reward: 1.000
18, 1: Disc Accuracy: 1.000, Aver